Lab4_Instrument_Control  
Author: Chengming Li  
All rights reserved  

In [1]:
import pyvisa
import numpy as np
from math import log10
from pathlib import Path
import shutil
from time import sleep
import matplotlib.pyplot as plt
import KS33600A_Control as FG
import KSE36311A_Control as PS
import KSMSOX3104G as OSC
PS_INSTRUMENT_ID = 'USB0::0x2A8D::0x1002::MY59301927::0::INSTR'
FG_INSTRUMENT_ID = 'USB0::0x0957::0x5707::MY59003236::0::INSTR'
OSC_INSTRUMENT_ID = 'USB0::0x2A8D::0x1770::MY63420135::0::INSTR'

In [6]:
""" List all instruments"""
rm = pyvisa.ResourceManager()
instruments = rm.list_resources()
print(instruments)

('USB0::0x0957::0x5707::MY59004729::0::INSTR', 'USB0::0x2A8D::0x1202::MY61005967::0::INSTR', 'USB0::0x2A8D::0x1770::MY63420113::0::INSTR', 'USB0::0x2A8D::0x1770::MY63420231::0::INSTR', 'USB0::0x0957::0x5707::MY59004869::0::INSTR')


Please follow the following steps for setup  
1. Power Supply Setup  
2. Function Generator Setup  
3. Oscilloscope Setup (Not done yet)  
4. Power Supply and Function Generator ON
5. !!! Sweep the measurement !!! (Not done yet)
6. Turn off Power Supply and Function Generator
7. Shut down all the remote control connection

In [2]:
""" 
    1. Power Supply Control
"""
ps = PS.PowerSupply_Connect(PS_INSTRUMENT_ID)

# Set channel 1 to 5 V, 1A
PS.Voltage_Setup(ps,1,2.5)
PS.Current_Setup(ps,1,0.2)

# Set channel 2 to 5 V, 1A
PS.Voltage_Setup(ps,2,5)
PS.Current_Setup(ps,2,0.2)

# Set channel 2 to 5 V, 1A
PS.Voltage_Setup(ps,3,-5)
PS.Current_Setup(ps,3,0.2)

Keysight Technologies,E36311A,MY59301927,2.1.0-1.0.4-1.12



In [11]:
""" 
    2. FunctionGenerator Control 
"""
start_freq = 1e3
start = log10(start_freq )
stop_freq = 10e3
stop = log10(stop_freq )
frequencies_SNR = np.logspace(start, stop, num=10, endpoint=True, base=10.0, dtype=None, axis=0)
# np.insert(frequencies_SNR, 0, 2e3)
# np.append(frequencies_SNR,250e3)

fg = FG.FuncGenConnect(FG_INSTRUMENT_ID)

"""
    Intended use channel 1 as SIN
    Set channel 1 to SIN, unit Vpp, HiZ Ohm, freq, amp, and offset
    Intended use channel 2 as CLK
    Set channel 2 to PWM, unit Vpp, 50 Ohm, freq, amp, and offset
"""
"""Channel1 1 Setup"""
FG.UnitVpp_setup(fg,1)
FG.Load__setup(fg,1,1)
freq = 1e3
amp = 1
offset = 0
FG.Sin_setup(fg,1,freq,amp,offset)


""" Channel 2 Setup"""
FG.UnitVpp_setup(fg,2)
FG.Load__setup(fg,2,0)
clk_freq = 1e6
clk_amp = 2.5
clk_offset = 1.25
clk_duty = 50
FG.PWM_setup(fg,2,clk_freq,clk_amp,clk_offset,clk_duty)
FG.Polarity_invert(fg,2,0)
FG.Sync_phase(fg)


# for freq_var in frequencies_SNR:
#     freq = freq_var
#     amp = 1
#     offset = 0
#     FG.Sin_setup(fg,1,freq,amp,offset)
#     print("frequency @" + str(freq))
#     sleep(5)

FG.output_OFF(fg,1)
FG.output_OFF(fg,2)


Agilent Technologies,33622A,MY59003236,A.02.03-3.15-03-64-02



In [ ]:
osc = OSC.Oscilloscope_Connect(OSC_INSTRUMENT_ID)
OSC.Oscilloscope_TimeBase(osc, 1/1e6)
osc.close()

In [5]:
""" 
    3. Oscilloscope Control 
"""
import KSMSOX3104G as OSC
import csv
import os
fg = FG.FuncGenConnect(FG_INSTRUMENT_ID)
FG.output_ON(fg,1)
FG.output_ON(fg,2)
FG.Sync_phase(fg)
osc = OSC.Oscilloscope_Connect(OSC_INSTRUMENT_ID)
OSC.Oscilloscope_SetBUS(osc)
OSC.Oscilloscope_RUN(osc)
OSC.DigitalDisplay_ON(osc,1)
OSC.Oscilloscope_Trigger(osc)
# OSC.Digitalizer_ON_OFF(osc, 0)
digi_th = 1.8
OSC.SetDigital_Threshold(osc,digi_th)
sleep(5)
# periodogram_length = 2^18
# num_segments = 8
# f_s = 1e6
# timebase = periodogram_length*num_segments/f_s

OSC.Oscilloscope_TimeBase(osc, 2/(1*1e3))
sleep(3)
OSC.Oscilloscope_Single(osc)
sleep(3)
OSC.Oscilloscope_Single(osc)
# OSC.Oscilloscope_Stop(osc)
################## Save Data from Oscilloscope
sleep(3)
[preamble, data] = OSC.Save_waveform(osc,"BUS1",)

combined_string = ''.join(data)

# Step 2: Split the string by the commas
rows = combined_string.split(',')
new_data = []
i = 0
for raw in rows[1:]:
    new_data.append(int(raw, 16))
    i = i+1

################## Create the foler
folder_path = r"C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\shikai-data"  # Change to your desired path
### change folder name as needed
folder_name = "11-27-Single-shikai"
file_path = os.path.join(folder_path, folder_name)
os.makedirs(file_path, exist_ok=True)

################## Create the file
# Sine Input parameters
sin_freq = 1e3
sin_amp = 1

# CLK Parameters
clk_freq = 100e3
clk_duty = 50

filename = "SlowerSample_"+str(int(sin_freq))+"Hz_" + str(int(sin_amp*1000))+"mV---CLK_"+ str(int(clk_freq/1e6)) + "MHz_" +str(int(clk_duty))+"%.csv"
file_path = os.path.join(file_path, filename)



with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write each integer as a row
    for number in new_data:
        writer.writerow([number]) 

print(f"Data saved to {file_path}")

OSC.Oscilloscope_RUN(osc)


FG.output_OFF(fg,1)
FG.output_OFF(fg,2)
osc.close()
fg.close()

Agilent Technologies,33622A,MY59003236,A.02.03-3.15-03-64-02

KEYSIGHT TECHNOLOGIES,MSO-X 3104G,MY63420135,07.60.2023080430

Data saved to C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\shikai-data\11-27-Single-shikai\SlowerSample_1000Hz_1000mV---CLK_0MHz_50%.csv


In [ ]:
""" 
    4. Please double check Your Power Supply and Function Generator and to match what you expect to see
"""
# Power Supply channel 1,2,3 ON
PS.Output_ON(ps,1)
PS.Output_ON(ps,2)
PS.Output_ON(ps,3)

# Function Generator channel 1 and 2 ON
FG.output_ON(fg,1)
FG.output_ON(fg,2)


In [19]:
""" 
    5. Sweep the Parameters ---- Prep Stage
"""
import KSMSOX3104G as OSC
import csv
import os
start_freq = 1e3
start = log10(start_freq )
stop_freq = 100e3
stop = log10(stop_freq)
frequencies_SNR = np.logspace(start, stop, num=30, endpoint=True, base=10.0, dtype=None, axis=0)
frequencies_SNR = np.linspace(start, stop, num=30)
np.insert(frequencies_SNR, 0, 2e3)
np.append(frequencies_SNR,250e3)

"""Instrument ID Setup"""
fg = FG.FuncGenConnect(FG_INSTRUMENT_ID)

osc = OSC.Oscilloscope_Connect(OSC_INSTRUMENT_ID)

ps = PS.PowerSupply_Connect(PS_INSTRUMENT_ID)


""" 
    1. Power Supply Control
"""

# Set channel 1 to 5 V, 1A
PS.Voltage_Setup(ps,1,2.5)
PS.Current_Setup(ps,1,0.2)

# Set channel 2 to 5 V, 1A
PS.Voltage_Setup(ps,2,5)
PS.Current_Setup(ps,2,0.2)

# Set channel 2 to 5 V, 1A
PS.Voltage_Setup(ps,3,-5)
PS.Current_Setup(ps,3,0.2)


"""
    Setup the Oscilloscope
"""

OSC.Oscilloscope_SetBUS(osc)
OSC.Oscilloscope_RUN(osc)
OSC.DigitalDisplay_ON(osc,1)
digi_th = 1.8
OSC.SetDigital_Threshold(osc,digi_th)
OSC.Oscilloscope_TimeBase(osc, 1/1e3)
# OSC.Oscilloscope_TimeBase(osc, 1/2e6)
sleep(3)

"""
    Setup the FunctionGenerator
"""
"""Channel1 1 Setup"""
FG.UnitVpp_setup(fg,1)
FG.Load__setup(fg,1,1)
freq = 1e3
amp = 1
offset = 0
FG.Sin_setup(fg,1,freq,amp,offset)


""" Channel 2 Setup"""
FG.UnitVpp_setup(fg,2)
FG.Load__setup(fg,2,0)
clk_freq = 1e6
clk_amp = 2.5
clk_offset = 1.25
clk_duty = 50
FG.PWM_setup(fg,2,clk_freq,clk_amp,clk_offset,clk_duty)
FG.Polarity_invert(fg,2,0)


FG.output_OFF(fg,1)
FG.output_OFF(fg,2)

FG.output_OFF(fg,1)
FG.output_OFF(fg,2)


Agilent Technologies,33622A,MY59003236,A.02.03-3.15-03-64-02

KEYSIGHT TECHNOLOGIES,MSO-X 3104G,MY63420135,07.60.2023080430

Keysight Technologies,E36311A,MY59301927,2.1.0-1.0.4-1.12



In [ ]:
PS.Output_ON(ps,1)
PS.Output_ON(ps,2)
PS.Output_ON(ps,3)
sleep(5)

for freq_var in frequencies_SNR:
    sin_freq = freq_var
    # !!! Change as needed
    # !!!!!!!!!!!!!!!!!!!!!!!
    sin_amp = 0.98
    sin_offset = 0
    FG.Sin_setup(fg,1,sin_freq,sin_amp,sin_offset)
    print("frequency @" + str(sin_freq))
    # Turn on THE CLK
    FG.output_ON(fg,2)
    sleep(5)
    OSC.Oscilloscope_Single(osc)

    ################## Save Data from Oscilloscope
    [preamble, data] = OSC.Save_waveform(osc,"BUS1",)
    combined_string = ''.join(data)

    # Step 2: Split the string by the commas
    rows = combined_string.split(',')
    new_data = []
    i = 0
    for raw in rows[1:]:
        new_data.append(int(raw, 16))
        i = i+1

    ################## Create the foler
    folder_path = r"C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\zhihao-data"  # Change to your desired path

    ### !!!!!!!!!change folder name as needed!!!!!!!!!!!!!
    folder_name = "11-26-SEDE-"+ str(int(sin_amp*1000))+"mV---CLK_"+ str(int(clk_freq/1e6)) + "MHz_" +str(int(clk_duty))
    file_path = os.path.join(folder_path, folder_name)
    os.makedirs(file_path, exist_ok=True)
    ################## Create the file
    filename = "SIN_"+str(int(sin_freq))+"Hz_" + str(int(sin_amp*1000))+"mV---CLK_"+ str(int(clk_freq/1e6)) + "MHz_" +str(int(clk_duty))+"%.csv"
    file_path = os.path.join(file_path, filename)



    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each integer as a row
        for number in new_data:
            writer.writerow([number]) 

    print(f"Data saved to {file_path}")

    OSC.Oscilloscope_RUN(osc)
    # FG.output_OFF(fg,2)
    FG.output_OFF(fg,1)
    sleep(2)


# folder_path = r"C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data"  # Change to your desired path

### !!!!!!!!!change folder name as needed!!!!!!!!!!!!!
# folder_name = "11-24-Python-Pull-Data_Sweep-1MHz-20%"
file_path = os.path.join(folder_path, folder_name)
os.makedirs(file_path, exist_ok=True)
################## Create the file
filename = "freq_array.csv"
file_path = os.path.join(file_path, filename)
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write each integer as a row
    for number in frequencies_SNR:
        writer.writerow([number]) 

print(f"Data saved to {file_path}")
FG.output_OFF(fg,1)
FG.output_OFF(fg,2)
PS.Output_OFF(ps,1)
PS.Output_OFF(ps,2)
PS.Output_OFF(ps,3)
osc.close()
fg.close()
ps.close()

frequency @1000.0
Data saved to C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\Board1\11-25-Python-Sweep-SEDE-1MHz-50%-1000mV-1k-100k-Test\SIN_1000Hz_1000mV---CLK_1MHz_50%.csv
frequency @1172.1022975334806
Data saved to C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\Board1\11-25-Python-Sweep-SEDE-1MHz-50%-1000mV-1k-100k-Test\SIN_1172Hz_1000mV---CLK_1MHz_50%.csv
frequency @1373.8237958832624
Data saved to C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\Board1\11-25-Python-Sweep-SEDE-1MHz-50%-1000mV-1k-100k-Test\SIN_1373Hz_1000mV---CLK_1MHz_50%.csv
frequency @1610.2620275609393
Data saved to C:\Users\steve\OneDrive\Documents\GitHub\ECE266_CMOSCircuitLab\Lab7_ICTesting\MatLabCode\ADC_Data\Board1\11-25-Python-Sweep-SEDE-1MHz-50%-1000mV-1k-100k-Test\SIN_1610Hz_1000mV---CLK_1MHz_50%.csv


KeyboardInterrupt: 

In [6]:
"""
    6. Turn off All power supply and function generator, and 
"""
# Power Supply channel 1,2,3 ON
PS.Output_OFF(ps,1)
PS.Output_OFF(ps,2)
PS.Output_OFF(ps,3)

# Function Generator channel 1 and 2 ON
FG.output_OFF(fg,1)
FG.output_OFF(fg,2)

In [ ]:
""" 
    7. Shut down the connection 
"""
ps.close()
fg.close()